In [23]:
filename="input.txt"
with open(filename) as f:
    lines = [line.rstrip().split(":") for line in f.readlines()]
    for i in range(len(lines)):
        line = lines[i]
        line[0] = line[0][10:]
        x_y = line[0].replace("x=","").replace("y=","").split(", ")
        line[0] = tuple(map(int,x_y))
        line[1] = tuple(map(int,line[1][22:].replace("x=","").replace("y=","").split(", ")))

sensors = [sensor[0] for sensor in lines]
beacons = [beacon[1] for beacon in lines]
combined = sensors+beacons

def print_map(sensors,beacons):
    combined = sensors+beacons
    min_x = min(combined, key=lambda tup: tup[0])[0]
    max_x = max(combined, key=lambda tup: tup[0])[0]
    min_y = min(combined, key=lambda tup: tup[1])[1]
    max_y = max(combined, key=lambda tup: tup[1])[1]

    for y in range(min_y,max_y):
        row_string = ""
        for x in range(min_x,max_x):
            if (x,y) in sensors:
                row_string += "S"
            elif (x,y) in beacons:
                row_string += "B"
            else:
                row_string += "."
        print(row_string)


In [41]:
#print("Map: ")
#print_map(sensors,beacons)
distances = []
for i in range(len(sensors)):
    distances.append(abs(sensors[i][0]-beacons[i][0])+abs(sensors[i][1]-beacons[i][1]))

min_x = min(combined, key=lambda tup: tup[0])[0]
max_x = max(combined, key=lambda tup: tup[0])[0]
#print(min_x,max_x)
row = 2000000
locations = 0 
row_string = ""

def manhatten(point1,point2):
    return abs(point1[0]-point2[0])+abs(point1[1]-point2[1])

all_positions = set([])
for i in range(len(sensors)):
    sensor = sensors[i]
    distance = distances[i]
    y_range = range(sensor[1]-distance,sensor[1]+distance)
    if row in y_range:
        intersection_distance = manhatten(sensor,(sensor[0],row))
        x_distance = distance-intersection_distance
        x_range = set(range(sensor[0]-x_distance,sensor[0]+x_distance+1))
        all_positions = all_positions.union(x_range)

for i in range(len(beacons)):
    x = beacons[i][0]
    if beacons[i][1] == row:
        if x in all_positions:
            all_positions.remove(x)

print(len(all_positions))
    
    

4961647


In [31]:
distances = []
for i in range(len(sensors)):
    distances.append(abs(sensors[i][0]-beacons[i][0])+abs(sensors[i][1]-beacons[i][1]))

locations = 0 
row_string = ""

def manhatten(point1,point2):
    return abs(point1[0]-point2[0])+abs(point1[1]-point2[1])


search_range = 4000000
full_set = set(range(search_range+1))

locations = 0 
row_string = ""

def manhatten(point1,point2):
    return abs(point1[0]-point2[0])+abs(point1[1]-point2[1])

for row in range(search_range+1):
    if row%100 == 0:
        print(f"Row {row}:")
    all_positions = set([])
    subset = False
    for i in range(len(sensors)):
        sensor = sensors[i]
        distance = distances[i]
        y_range = range(sensor[1]-distance,sensor[1]+distance+1)
        if row in y_range:
            intersection_distance = manhatten(sensor,(sensor[0],row))
            x_distance = distance-intersection_distance
            min_x = sensor[0]-x_distance
            max_x = sensor[0]+x_distance+1
            if min_x > search_range or max_x < 0:
                print("Why")
            if min_x < 0:
                min_x = 0
            if max_x > search_range:
                max_x = search_range+1
            

            x_range = set(range(min_x,max_x))
            all_positions = all_positions.union(x_range)
            if full_set == all_positions:
                 #print("FullSet is Subset")
                 subset = True
                 break
    # if not subset:
        # print("ye")
        # print(all_positions)
        # print(full_set)
        # print(full_set.issubset(all_positions))
    #reduced = full_set.intersection(all_positions)
    if not subset:
        difference = full_set.difference(all_positions)
        if len(difference) > 0:
            print(list(difference)[0]*4000000+row)
            break



Row 0:
108000000


In [24]:
import itertools

distances = []
for i in range(len(sensors)):
    distances.append(abs(sensors[i][0]-beacons[i][0])+abs(sensors[i][1]-beacons[i][1]))

locations = 0 
row_string = ""

# def manhatten(point1,point2):
#     return abs(point1[0]-point2[0])+abs(point1[1]-point2[1])


search_range = 4000000
full_set = set(range(search_range+1))

locations = 0 
row_string = ""

def manhatten(point1,point2):
    return abs(point1[0]-point2[0])+abs(point1[1]-point2[1])

def overlap(a, b):
    return a[-1] > b[0] and a[0] < b[-1]

def compare_ranges(r1,r2):
    #print("Compareing: ",r1[0],r2[0],r1[-1],r2[0])
    return max(r1[0],r2[0]) <= min(r1[-1],r2[-1])
    #return r1[-1] >= r2[0]

def valid_range(r1,r2):
    #print("In overlap: ",r2[-1]+1,r2[-1])
    if r1[-1]+1 < r2[0]:
        return False
    if r1[0]>=r2[0] and r2[-1] <= r2[0]:
        #print("First Case")
        return r2
    elif r1[0] <= r2[0] and r2[-1] >= r2[-1]:
        #print("Second Case")
        return r1

    elif r1[-1] >= r2[0] and r1[0] <= r2[0]:
        #print("Third Case")
        return range(r1[0],r2[-1])

    elif r1[-1]+1 == r2[0] and r1[0] <= r2[0]:
        #print("Fourth Case")
        return range(r1[0],r2[-1])

found = False
for row in range(search_range+1):
    #print(f'Row {row}:')
    if row%100000 == 0:
        print(f"Row {row}:")
    all_positions = full_set
    subset = False
    ranges = []
    for i in range(len(sensors)):
        sensor = sensors[i]
        distance = distances[i]
        y_range = range(sensor[1]-distance,sensor[1]+distance+1)
        if row in y_range:
            intersection_distance = manhatten(sensor,(sensor[0],row))
            x_distance = distance-intersection_distance
            min_x = sensor[0]-x_distance
            max_x = sensor[0]+x_distance+1
            if min_x < 0:
                min_x = 0
            if max_x > search_range:
                max_x = search_range+1
            x_range = range(min_x,max_x)
            # if len(x_range) == 0:
            #     print(x_range)
            ranges.append(x_range)

    ranges = sorted(ranges, key=lambda r: (r[0],r[-1]))
    #print(ranges)
    cur_range = range(0)
    for i in range(len(ranges)-1):
        if i == 0:
            cur_range = ranges[i]
        comp_range = ranges[i+1]
        #print("Cur Range: ",cur_range," Comp Range: ",comp_range)
        # print(cur_range)
        # print(comp_range)
        # print(compare_ranges(cur_range,comp_range))
        if valid_range(cur_range,comp_range):
            cur_range = range(0, max(cur_range[-1],comp_range[-1])+1)
            
        else:
            found = True
            break
    if found:
        print("Yay")
        print(((cur_range[-1]+1)*4000000)+row)
        break
        
        
    

        
    #print(ranges)
            # all_positions = all_positions.difference(x_range)
            # if len(all_positions) == 0:
            #      #print("FullSet is Subset")
            #      subset = True
            #      break
    # if not subset:
        # print("ye")
        # print(all_positions)
        # print(full_set)
        # print(full_set.issubset(all_positions))
    #reduced = full_set.intersection(all_positions)
    # if not subset:
    #     print(list(all_positions)[0]*4000000+row)
    #     break
        # difference = full_set.difference(all_positions)
        # if len(difference) > 0:
        #     print(list(difference)[0]*4000000+row)
        #     break

Row 0:
Yay
56000011
